In [1]:
import psycopg2
import pandas as pd
import numpy as np

In [2]:
dbname = 'podcast'
username = 'lindsay'

In [3]:
similarity_df = pd.read_pickle('gensim/similarity_df.pkl')

In [4]:
con = psycopg2.connect(database = dbname, user = username)
cursor = con.cursor()

In [5]:
# unstack each list
match_id = similarity_df.match_id.apply(lambda x: pd.Series(x)).unstack()
search_id = similarity_df.search_id.apply(lambda x: pd.Series(x)).unstack()
similarity = similarity_df.similarity.apply(lambda x: pd.Series(x)).unstack()

In [10]:
# put unstacked lists in new df
similarity_unstacked = pd.DataFrame({'search_id' : search_id,
                                     'match_id' : match_id,
                                     'similarity' : similarity})

In [16]:
similarity_unstacked['match_id'] = [int(x) for x in similarity_unstacked['match_id']]

In [21]:
similarity_unstacked['similarity'] = [x.item() for x in similarity_unstacked['similarity']]

In [22]:
similarity_unstacked.dtypes

match_id        int64
search_id       int64
similarity    float64
dtype: object

In [23]:
# insert similarity data 
query = "INSERT INTO podcast_similarity (search_id, match_id, similarity) VALUES (%s, %s, %s);"

for ind, row in similarity_unstacked.iterrows():
    data = (row['search_id'], row['match_id'], row['similarity'])
    cursor.execute(query, data)
    
con.commit()